# Wykład dotyczący przypomnienia podstaw numerycznej algebry liniowej

Plan:

* normy wektorów i macierzy
* współczynnik uwarunkowania
* eliminacja Gaussa
* wsteczne podstawienie (do przodu i w tył)
* eliminacja Gaussa-Jordana
* faktoryzacja LU
  * metoda Doolitle'a
  * metoda eliminacji Gaussa
* selekcja elementu głównego


In [11]:
using LinearAlgebra

Przykład współczynnika uwarunkowania.

In [12]:
A = [1 2; 2 3.999]'
b = [4, 7.999]
c = cond(A) 
println("cond(A) = $c")  # bardzo duuuży!
println("|A^(-1)|*|A| = $(norm(inv(A), 2)*norm(A, 2))")
x1 = A\b  # wynik to ~ [2,0] 

cond(A) = 24992.00096006945
|A^(-1)|*|A| = 24992.001000002758


2-element Array{Float64,1}:
 1.9999999999991118
 1.000000000000444

In [13]:
b = [4,8]
x2 = A\b

2-element Array{Float64,1}:
 4.0
 0.0

In [14]:
delta_b = 0.001 / norm(b)
delta_x = c * delta_b

2.7941906520227646

In [15]:
rzeczywiste_delta_x = norm(x2-x1) / norm(x2)

0.5590169943751957

Zmieniamy wartość `A[2,2]` macierzy z 3.999 na 3.

In [16]:
A = [1 2; 2 3]'
b = [4, 7.999]
c = cond(A) 
println("cond(A) = $c")  # bardzo duuuży!
println("|A^(-1)|*|A| = $(norm(inv(A), 2)*norm(A, 2))")
x1 = A\b  # wynik to ~ [2,0] 

cond(A) = 17.944271909999173
|A^(-1)|*|A| = 17.999999999999996


2-element Array{Float64,1}:
 3.9979999999999993
 0.001000000000000334

In [17]:
b = [4,8]
x2 = A\b

2-element Array{Float64,1}:
 4.0
 0.0

In [18]:
delta_b = 0.001 / norm(b)
delta_x = c * delta_b

0.0020062305898749066

In [19]:
rzeczywiste_delta_x = norm(x2-x1) / norm(x2)

0.0005590169943751341

In [20]:
?norm

search: norm normpath normalize normalize! normalize_hue opnorm issubnormal



```
norm(A, p::Real=2)
```

For any iterable container `A` (including arrays of any dimension) of numbers (or any element type for which `norm` is defined), compute the `p`-norm (defaulting to `p=2`) as if `A` were a vector of the corresponding length.

The `p`-norm is defined as

$$
\|A\|_p = \left( \sum_{i=1}^n | a_i | ^p \right)^{1/p}
$$

with $a_i$ the entries of $A$, $| a_i |$ the [`norm`](@ref) of $a_i$, and $n$ the length of $A$. Since the `p`-norm is computed using the [`norm`](@ref)s of the entries of `A`, the `p`-norm of a vector of vectors is not compatible with the interpretation of it as a block vector in general if `p != 2`.

`p` can assume any numeric value (even though not all values produce a mathematically valid vector norm). In particular, `norm(A, Inf)` returns the largest value in `abs.(A)`, whereas `norm(A, -Inf)` returns the smallest. If `A` is a matrix and `p=2`, then this is equivalent to the Frobenius norm.

The second argument `p` is not necessarily a part of the interface for `norm`, i.e. a custom type may only implement `norm(A)` without second argument.

Use [`opnorm`](@ref) to compute the operator norm of a matrix.

# Examples

```jldoctest
julia> v = [3, -2, 6]
3-element Array{Int64,1}:
  3
 -2
  6

julia> norm(v)
7.0

julia> norm(v, 1)
11.0

julia> norm(v, Inf)
6.0

julia> norm([1 2 3; 4 5 6; 7 8 9])
16.881943016134134

julia> norm([1 2 3 4 5 6 7 8 9])
16.881943016134134

julia> norm(1:9)
16.881943016134134

julia> norm(hcat(v,v), 1) == norm(vcat(v,v), 1) != norm([v,v], 1)
true

julia> norm(hcat(v,v), 2) == norm(vcat(v,v), 2) == norm([v,v], 2)
true

julia> norm(hcat(v,v), Inf) == norm(vcat(v,v), Inf) != norm([v,v], Inf)
true
```

---

```
norm(x::Number, p::Real=2)
```

For numbers, return $\left( |x|^p \right)^{1/p}$.

# Examples

```jldoctest
julia> norm(2, 1)
2.0

julia> norm(-2, 1)
2.0

julia> norm(2, 2)
2.0

julia> norm(-2, 2)
2.0

julia> norm(2, Inf)
2.0

julia> norm(-2, Inf)
2.0
```

---

```
norm(h::Histogram)
```

Calculate the norm of histogram `h` as the absolute value of its integral.


In [21]:
function gaussian(A,b)
  Ag = [A b]
  n = size(Ag,1);
  for k=1:n-1
    for i = k+1:n
      l = Ag[i,k] / Ag[k,k];
      for j = k:n+1
        Ag[i,j] = Ag[i,j] - l * Ag[k,j]; 
      end
    end
  end   
  return Ag
end

gaussian (generic function with 1 method)

In [22]:
function backward(U,c)
    n = size(U,1);
    x = zeros(n,1);
    for i = n:-1:1
        s = 0;
        for j = i+1:n
            s = s + U[i,j]*x[j];
        end
        x[i] = (c[i] - s) / U[i,i];
    end
    return x
end 

backward (generic function with 1 method)

In [23]:
function forward(L,c)
    n = size(L,1);
    x = zeros(n,1);
    for i = 1:n
        s = 0;
        for j = 1:i-1
            s = s + L[i,j]*x[j];
        end
        x[i] = (c[i] - s) / L[i,i];
    end
    return x
end

forward (generic function with 1 method)

Na slajdach zastosowaliśmy bardzo grube zaokrąglenie! Tutaj wynik jest właściwy ze względu na bardzo dużą precyzję i niewielką macierz.

In [24]:
A = [0.003  59.1;
5.291  -6.13 ]
b = [59.17;
     46.78]
Ag = gaussian(A,b)
x = backward(Ag[:, 1:end-1], Ag[:,end])

2×1 Array{Float64,2}:
 10.000784096803272
  1.0006767791490625

In [25]:
function gauss_jordan(A,b)
  Ag = [A b]
  n = size(Ag,1);
  for k=1:n
    Ag[k,:] = Ag[k,:] / Ag[k,k];
    for i = 1:n
      if k != i
          l = Ag[i,k] / Ag[k,k];
          Ag[i,k:end] = Ag[i,k:end] - l * Ag[k,k:end];       
      end
    end
  end
  return Ag
end

gauss_jordan (generic function with 1 method)

In [26]:
A = [0.003  59.1;
5.291  -6.13 ]
b = [59.17;
     46.78]
Ag = gauss_jordan(A,b)

2×3 Array{Float64,2}:
  1.0  0.0  10.0008
 -0.0  1.0   1.00068

A teraz spróbujemy obliczyć macierz odwrotną.

Zamiast wektora prawych stron przekażemy macierz jednostkową.

In [27]:
Ag = gauss_jordan(A,I)

2×4 Array{Float64,2}:
  1.0  0.0  0.0196024   0.188989
 -0.0  1.0  0.0169195  -9.59335e-6

## Dekompozycja LU (faktoryzacja, rozkłąd na czynniki)

In [28]:
function eye(n)
    return Matrix{Float64}(I,n,n)
end

eye (generic function with 1 method)

In [29]:
function doolitle(A)
    n = size(A,1);
    L = eye(n);
    U = zeros(n,n);
    U[1,:] = A[1,:]
    L[2:n,1] = A[2:n,1] / U[1,1];
    
    for i = 2:n
        for k = i:n
            s = 0;
            for j=1:i-1
                s = s + L[i,j]*U[j,k]
            end
            U[i,k] = A[i,k] - s
        end
        for k = i+1:n
            s = 0
            for j=1:i-1
                s = s + L[k,j]*U[j,i];
            end
            L[k,i] = (A[k,i] - s) / U[i,i];
        end
    end   
    return L,U
end

doolitle (generic function with 1 method)

In [30]:
A = [1 -1 1 1
    4 3 -1 2
    3 2 2 5
    8 9 5 8];
b = [4 6 15 1]'
L, U = doolitle(A)
display(L)
display(U)
display(A - L*U)  # powinno zwrócić macierz bliską zeru

4×4 Array{Float64,2}:
 1.0  0.0       0.0      0.0
 4.0  1.0       0.0      0.0
 3.0  0.714286  1.0      0.0
 8.0  2.42857   3.55556  1.0

4×4 Array{Float64,2}:
 1.0  -1.0   1.0       1.0
 0.0   7.0  -5.0      -2.0
 0.0   0.0   2.57143   3.42857
 0.0   0.0   0.0      -7.33333

4×4 Array{Float64,2}:
 0.0  0.0          0.0          0.0
 0.0  0.0          0.0          0.0
 0.0  0.0          0.0          0.0
 0.0  1.77636e-15  8.88178e-16  0.0

Dekompozycja LU metodą eliminacji Gaussa

In [31]:
function lu_gaussian(A)
    n = size(A, 1);
    L = eye(n);
    for c = 1:n-1
        for r = c+1:n            
            f = A[r,c] / A[c,c];
            A[r, :] = A[r, :] - f*A[c,:];
            L[r, c] = f;
        end
    end
    return L,A
end

lu_gaussian (generic function with 1 method)

In [32]:
A = rand(4,4)
L, U = lu_gaussian(copy(A))
display(A)
display(L)
display(U)
display(L * U - A)
norm(L * U - A, 2)

4×4 Array{Float64,2}:
 0.0253537  0.872085  0.0622252  0.572571
 0.812664   0.494769  0.463935   0.724954
 0.524582   0.400107  0.722851   0.226687
 0.709835   0.139847  0.836546   0.0813297

4×4 Array{Float64,2}:
  1.0     0.0       0.0      0.0
 32.0531  1.0       0.0      0.0
 20.6905  0.642568  1.0      0.0
 27.9973  0.884112  1.06859  1.0

4×4 Array{Float64,2}:
 0.0253537    0.872085   0.0622252      0.572571
 0.0        -27.4582    -1.53058      -17.6277
 0.0          0.0        0.418877      -0.293105
 0.0          0.0       -5.55112e-17   -0.0510218

4×4 Array{Float64,2}:
 0.0   0.0          0.0   0.0
 0.0  -1.11022e-15  0.0   6.66134e-16
 0.0  -4.996e-16    0.0  -1.11022e-15
 0.0  -1.77636e-15  0.0   3.88578e-16

2.5426288831284585e-15

## Przykład obliczeniowy

In [33]:
A = [3.0 1 6
     1 1 1
     2 1 3]
b = [4
     6
     15]

L,U = lu_gaussian(copy(A))
display(L)
display(U)

y = forward(L,b)
x = backward(U,y)
display(x)

display(norm(b - A*x)) # spodziewamy się 0

3×3 Array{Float64,2}:
 1.0       0.0  0.0
 0.333333  1.0  0.0
 0.666667  0.5  1.0

3×3 Array{Float64,2}:
 3.0  1.0        6.0
 0.0  0.666667  -1.0
 0.0  0.0       -0.5

3×1 Array{Float64,2}:
  49.0
 -22.999999999999996
 -20.0

3.552713678800501e-15

## Elimnacja Gaussa z selekcją elementu głównego

In [34]:
function gaussian_pivot(A,b)
  Ag = [A b]
  n = size(Ag,1);
  for k=1:n-1
    for i = k+1:n
      idx = argmax(Ag[k:end,k]) + k - 1
      tmp = Ag[idx,:]
      Ag[idx,:] = Ag[k,:]    
      Ag[k,:] = tmp
            
      l = Ag[i,k] / Ag[k,k];
      Ag[i,k:end] = Ag[i,k:end] - l * Ag[k,k:end]; 
    end
  end   
  return Ag
end

gaussian_pivot (generic function with 1 method)

In [47]:
A = [0.1 1 2
     0 4 -2
     3 0 3]
b = [1
     2
     3]
Ag = gaussian_pivot(A,b)
x = backward(Ag[:,1:end-1], Ag[:,end])
display("Rozwiązanie:")
display(x)

norm(A*x-b) # oczekujemy 0

"Rozwiązanie:"

3×1 Array{Float64,2}:
 0.8333333333333334
 0.5833333333333334
 0.16666666666666669

0.0